<a href="https://colab.research.google.com/github/SeokcheonMoon/study_data_analytics/blob/main/docs/quests/CDAs/CDA_continuous.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 한글 폰트 설치
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 설치 후 colab의 경우 Runtime 이후 상단바 런타임의 Restart Session 을 클릭해야함
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용
import scipy
import scipy.stats as stats

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 32 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 2s (5,294 kB/s)
Selecting previously unselected package fonts-nanum.
(Reading database ... 121747 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr

In [ ]:
df_fruitstore = pd.read_csv("/content/drive/MyDrive/fruitstore_saleshistory.csv")

In [ ]:
df_fruitstore

,공급일자,요일,공급월,공급주차,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,주소-구,주소-동,성별,연령,연령대,구매금액,구매매장,반품_원거래일자,구매시각
0,2018-01-02,화,1,1,272369856,정상회원,과실,과일,사과,사과/유(1.5kg),1.0,수지구,풍덕천동,여,45.0,40대,22207,매장C,NaN,10:04
1,2018-01-02,화,1,1,1506656256,정상회원,채소,버섯,느타리버섯,느타리버섯(300g),1.0,수지구,풍덕천동,여,36.0,30대이하,4977,매장C,NaN,10:05
2,2018-01-02,화,1,1,1506656256,정상회원,축산물,알,유정란,유정란/매장용(10알/국내산),1.0,수지구,풍덕천동,여,36.0,30대이하,7083,매장C,NaN,10:05
3,2018-01-02,화,1,1,1023108864,정상회원,반찬,두부/유부,두부,연두부(100g),1.0,수지구,풍덕천동,여,36.0,30대이하,766,매장C,NaN,10:08
4,2018-01-02,화,1,1,1476143616,정상회원,간식,빵,"식빵, 식사대용",아침빵(240g/8개)/매장,1.0,수지구,풍덕천동,여,34.0,30대이하,4403,매장C,NaN,10:09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705566,2018-06-30,토,6,26,146297344,정상회원,수산,건어물,멸치류,국물멸치(300g),1.0,수지구,성복동,여,51.0,50대,9189,매장A,NaN,17:56
705567,2018-06-30,토,6,26,1532985472,정상회원,수산,해조,미역/다시마,자른다시마(100g),1.0,수지구,동천동,여,63.0,60대,5169,매장A,NaN,14:37
705568,2018-06-30,토,6,26,1532985472,정상회원,반찬,두부/유부,두부,두부(1모:420g),2.0,수지구,동천동,여,63.0,60대,7466,매장A,NaN,14:37
705569,2018-06-30,토,6,26,1209779776,정상회원,간식,과자,스낵,하늘바다새우(60g),1.0,수지구,동천동,여,41.0,40대,3542,매장A,NaN,18:04


## 가설 2-4 (분석내용): 고객 충성도 점수(X)가 평균 거래 가치(Y)를 예측합니까?

- 귀무가설 : 고객 충성도 점수는 평균 거래 가치를 예측하지 못한다.(관련이 없다.) (p>0.05)
- 대립가설 : 고객 충성도 점수는 평균 거래 가치를 예측한다.(관련이 있다.) (p<0.05)
- 평균거래가치 => 각 고객의 평균거래가치 라고 생각을 한 후 분석.

---



### 충성도 고객 파악 (총 구매금액을 기반)

In [ ]:
# 데이터프레임에서 회원번호와 구매금액 컬럼 선택
df_subset = df_fruitstore[['회원번호', '구매금액']]

# 회원번호별 구매금액의 합계 계산
df_sum = df_subset.groupby('회원번호')['구매금액'].sum()

# 결과 출력
df_sum

회원번호
18764160        604284
18792000         70642
18942336       1010616
19391488          3159
19627200       4101241
                ...   
1607535424       13784
1607672768      121949
1670400000    38171716
1670606016      225230
1670620864      820999
Name: 구매금액, Length: 11202, dtype: int64

In [ ]:
df_sum.dropna()

회원번호
18764160        604284
18792000         70642
18942336       1010616
19391488          3159
19627200       4101241
                ...   
1607535424       13784
1607672768      121949
1670400000    38171716
1670606016      225230
1670620864      820999
Name: 구매금액, Length: 11202, dtype: int64

In [ ]:
df_sum_sorted = pd.DataFrame(df_sum.sort_values(ascending = False)[:5])
df_sum_sorted

,구매금액
회원번호,
1670400000,38171716
436681536,22549159
381879424,17294603
635659584,16175518
444617792,15037518


In [ ]:
df_sum_sorted["구매횟수"] =  df_fruitstore["회원번호"].value_counts()

1670400000    3586
436681536     1362
444617792     1166
548171456     1155
72796032      1120
              ... 
1160536384       1
401603136        1
1579504256       1
1202006848       1
26440576         1
Name: 회원번호, Length: 11202, dtype: int64

In [ ]:
38171716 / 3586

10644.650306748466

In [ ]:
condition_first = "회원번호 == 1670400000 "
df_fruitstore_first = df_fruitstore.query(condition_first)
df_fruitstore_first

,공급일자,요일,공급월,공급주차,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,주소-구,주소-동,성별,연령,연령대,구매금액,구매매장,반품_원거래일자,구매시각
448,2018-01-02,화,1,1,1670400000,탈퇴,음료,음료,과즙,배즙(120ml/1봉-매장용),2.0,기타,비조합원,남,36.0,30대이하,4212,매장B,NaN,17:12
1155,2018-01-02,화,1,1,1670400000,탈퇴,축산물,유제품,요구르트,떠먹는 요거트(90g*2개),1.0,기타,비조합원,남,36.0,30대이하,7370,매장A,NaN,11:07
1156,2018-01-02,화,1,1,1670400000,탈퇴,축산물,유제품,요구르트,달지않은 떠먹는요거트(90g*2개),1.0,기타,비조합원,남,36.0,30대이하,7370,매장A,NaN,11:07
3034,2018-01-02,화,1,1,1670400000,탈퇴,축산물,유제품,우유/산양유,우유(범산/900ml),1.0,기타,비조합원,남,36.0,30대이하,10108,매장A,NaN,11:07
3590,2018-01-02,화,1,1,1670400000,탈퇴,양념/가루,장,된장,우리콩된장(900g),1.0,기타,비조합원,남,36.0,30대이하,40011,매장B,NaN,16:05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703542,2018-06-30,토,6,26,1670400000,탈퇴,과실,과일,참다래,참다래(그린:2kg),1.0,기타,비조합원,남,36.0,30대이하,23930,매장D,NaN,10:56
704119,2018-06-30,토,6,26,1670400000,탈퇴,과실,과일,자두,자두/대석외/소(2kg),1.0,기타,비조합원,남,36.0,30대이하,12827,매장D,NaN,10:56
704310,2018-06-30,토,6,26,1670400000,탈퇴,채소,잎/줄기채소,브로컬리,브로컬리(250g),1.0,기타,비조합원,남,36.0,30대이하,3982,매장A,NaN,11:04
704525,2018-06-30,토,6,26,1670400000,탈퇴,반찬,두부/유부,두부,두부(1모:420g),2.0,기타,비조합원,남,36.0,30대이하,7466,매장C,NaN,18:10


In [ ]:
df_fruitstore["구매금액"].sum()
#22549159       #2
#16555.91703377386
\
#38171716   #1
#10644.650306748466

7704862190

In [ ]:
df_fruitstore_conditions_category_pivot = pd.pivot_table(df_fruitstore_conditions[["물품중분류", "요일"]], index = "물품중분류", columns = ["요일"], aggfunc = len)
df_fruitstore_conditions_category_pivot

NameError: name 'df_fruitstore_conditions' is not defined

### 분석결과
- 통계결과
- 사실 결론 :